# Class Weight Compution

In [2]:
import math
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
# export_ds = '/media/scratch/berlukas/nuscenes'

# sem_clouds_filename = f"{export_ds}/sem_bkp/sem_classes_gt1.npy"
sem_clouds_filename = f"{export_ds}/sem_clouds2.npy"

print(f"Loading sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename).astype(int)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading sem clouds from /media/berlukas/Data2/datasets/nuscenes/processed//sem_clouds2.npy
Shape of sem clouds is (11452, 3, 200, 200)


In [3]:
export_ds = '/media/scratch/berlukas/nuscenes/training'
samples = os.listdir(export_ds)
sem_cloud_features = None
for sample in samples:
    sem_clouds_filename = f"{export_ds}/{sample}"    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename).astype(int)
    else:
        features = np.load(sem_clouds_filename).astype(int)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_waymo_00_11_bw120.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_02_kitti_bw120.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_05_kitti_bw120.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_nuscenes1_bw120.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_00_kitti_bw120.npy
Shape of sem clouds is (31072, 2, 240, 240)


In [4]:
n_classes = 7
class_counts = [0] * n_classes
for cl in range(1, n_classes): 
    count = np.count_nonzero(sem_cloud_features[:,1,:,:] == cl)
    print(f'Computing occurrences for {cl}: {count}')
    class_counts[cl] = count

Computing occurrences for 1: 20758609
Computing occurrences for 2: 779174
Computing occurrences for 3: 68154113
Computing occurrences for 4: 48083923
Computing occurrences for 5: 52855520
Computing occurrences for 6: 47849007


In [5]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)
rates = class_counts / total_counts
print(f'Should sum to one: {np.sum(rates) == 1.0}')
print(f'\nRates are\n====================\n')
print(f'Vehicle: {rates[1]}')
print(f'Living: {rates[2]}')
print(f'Road: {rates[3]}')
print(f'Ground: {rates[4]}')
print(f'Manmade: {rates[5]}')
print(f'Vegetation: {rates[6]}')



inv_rates = 1-rates
# inv_rates[0] = 0
# inv_rates[8] = 0
# weights = np.round_((inv_rates) / np.sum(inv_rates), 20)

rates = rates[1:]
weights = np.power(np.amax(rates) / rates, 1 / 3.0)

print(f'\nWeights are\n====================\n')
print(f'Vehicle: {weights[0]}')
print(f'Living: {weights[1]}')
print(f'Road: {weights[2]}')
print(f'Ground: {weights[3]}')
print(f'Manmade: {weights[4]}')
print(f'Vegetation: {weights[5]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {weights}')

Counts are 
[0, 20758609, 779174, 68154113, 48083923, 52855520, 47849007]
Should sum to one: True

Rates are

Vehicle: 0.08704536599422746
Living: 0.0032672461822074008
Road: 0.28578503068760225
Ground: 0.20162635540624382
Manmade: 0.2216347002448579
Vegetation: 0.20064130148486115

Weights are

Vehicle: 1.4862707824870212
Living: 4.439007436362647
Road: 1.0
Ground: 1.1233042248939462
Manmade: 1.088430174964464
Vegetation: 1.1251395222200713


Weights sum to one: False (10.262152140928148)
weights: [1.48627078 4.43900744 1.         1.12330422 1.08843017 1.12513952]


In [ ]:
mu = sem_cloud_features[:,0,:,:].mean()
sigma = sem_cloud_features[:,0,:,:].std()
print(f'We have a mean of {mu} and a std of {sigma}')

In [11]:
print(f'We have a mean of {mu} and a std of {sigma}')

We have a mean of 1.2839795737632667 and a std of 7.69180843049593
